In [1]:
from torch.utils.data import DataLoader
import torch.nn as nn
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn
import torch
import os

print("pd - ", pd.__version__) 
print("sklearn - ", sklearn.__version__) 
print("np - ", np.__version__) 
print("sns - ", sns.__version__) 
print("sklearn - ", sklearn.__version__) 
print("torch - ", torch.__version__)

pd -  2.0.3
sklearn -  1.5.1
np -  1.23.5
sns -  0.13.2
sklearn -  1.5.1
torch -  2.4.1+cu118


In [2]:
np.random.seed(123)  # Semente da aleatoriedade.
torch.manual_seed(123)  # Para os pesos da rede começar sempre iguais (BOA PRÁTICA).
torch.cuda.manual_seed(123)

baseFolder = os.path.join(os.path.dirname(os.getcwd()),"Bases")

previsoresData = os.path.join(baseFolder, "entradas_breast.csv")
classeData = os.path.join(baseFolder, "saidas_breast.csv")

previsores = pd.read_csv(previsoresData)  # Caracteristicas do tumor
classe = pd.read_csv(classeData)  # Maligno ou benigno

In [3]:
previsores = torch.tensor(np.array(previsores), dtype=torch.float)
classes = torch.tensor(np.array(classe), dtype=torch.float)

In [4]:
import torch.utils
import torch.utils.data

train_loader = DataLoader(torch.utils.data.TensorDataset(previsores, classes),
                          batch_size=10, 
                          shuffle=True)

In [5]:
# Construção do modelo
class classificador_torch(nn.Module):
    
    def __init__(self) -> None:
        super().__init__()
        
        # Inicialização dos pesos
        self.dense0 = torch.nn.Linear(30, 8)
        torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std=0.05)  # Faz a inicialização dos pesos levando em ocnsideração o desvio.
        self.dense1 = torch.nn.Linear(8, 8)
        torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std=0.05)
        self.dense2 = torch.nn.Linear(8, 1)
        self.activation = torch.nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.dense0(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.dense1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.dense2(x)
        output = self.output(x)
        
        return output
    

In [6]:
classificador = classificador_torch()

In [7]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classificador.parameters(), lr=0.001, weight_decay=0.0001)

In [8]:
# Treinamento do modelo.
for epoch in range(150):
    
    runnin_loss = 0
    
    for data in train_loader:
        
        inputs, labels = data
        optimizer.zero_grad()  # Zera gradiente 
        
        outputs = classificador(inputs)
        
        loss = criterion(outputs, labels)  # Pega erro com base na predição e dados reais
        loss.backward()
        optimizer.step()
        
        runnin_loss += loss.item()
        
    print(f"ÉPOCH {epoch + 1}, {runnin_loss/ len(train_loader)}")

ÉPOCH 1, 16.843043593591766
ÉPOCH 2, 1.8109540879203563
ÉPOCH 3, 0.7080962501074138
ÉPOCH 4, 0.670868156249063
ÉPOCH 5, 0.6516150497553641
ÉPOCH 6, 0.6161274690377084
ÉPOCH 7, 0.6120375598731794
ÉPOCH 8, 0.5925950977885932
ÉPOCH 9, 0.5915186033959974
ÉPOCH 10, 0.5578441002912689
ÉPOCH 11, 0.5748710710751382
ÉPOCH 12, 0.5793091424724512
ÉPOCH 13, 0.5886205063577283
ÉPOCH 14, 0.5657687176737869
ÉPOCH 15, 0.5801995495955149
ÉPOCH 16, 0.5666103661060333
ÉPOCH 17, 0.5705412631494957
ÉPOCH 18, 0.5876957470910591
ÉPOCH 19, 0.5645796635694671
ÉPOCH 20, 0.5616445645951388
ÉPOCH 21, 0.5617436980992033
ÉPOCH 22, 0.5573642478700269
ÉPOCH 23, 0.5532874773468888
ÉPOCH 24, 0.5574886788401687
ÉPOCH 25, 0.5649689237276713
ÉPOCH 26, 0.5569732586542765
ÉPOCH 27, 0.5485042270861173
ÉPOCH 28, 0.5368118803752097
ÉPOCH 29, 0.5520796247741633
ÉPOCH 30, 0.5505548062031729
ÉPOCH 31, 0.5490319258288333
ÉPOCH 32, 0.5403958140758046
ÉPOCH 33, 0.5445332898382556
ÉPOCH 34, 0.5446286949149349
ÉPOCH 35, 0.530074651826

In [9]:
# Pesos da rede
classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-4.9899e-03,  2.3418e-04, -5.1511e-03, -1.2797e-03, -6.5552e-02,
                        5.7504e-03,  3.1799e-02, -1.4916e-02, -4.8228e-02, -2.0683e-02,
                       -3.8868e-03, -1.2680e-02, -2.1246e-02,  4.1091e-02, -2.2961e-17,
                        1.6953e-16,  9.6646e-18, -4.3198e-12,  7.4416e-03,  4.6670e-17,
                        3.4795e-02, -8.0554e-02, -5.5831e-02, -2.6519e-02, -1.0744e-04,
                       -2.4288e-03,  3.7040e-03, -1.2666e-02,  1.2023e-02, -1.7501e-03],
                      [-4.8514e-01, -3.3041e-01, -3.7431e-01, -7.0810e-02,  1.2922e-01,
                        1.1530e-01,  1.1160e-01, -1.4833e-01, -9.2875e-03,  1.0691e-01,
                       -9.3235e-03,  2.8159e-04,  3.5821e-03, -2.6402e-02,  4.6819e-03,
                       -1.5240e-01,  4.0917e-02, -2.1008e-01,  4.7321e-01,  1.4853e-01,
                       -3.4961e-01, -2.9083e-01, -2.9016e-01,  1.6122e-01,  2.0231e-03,


In [10]:
torch.save(classificador.state_dict(), "classificador.pth")